In [24]:
import pandas as pd
import re 
from datasets import Dataset

# Load the Dataset

In [2]:
data_path = "../data/naruto.csv"
naruto_transcript_df = pd.read_csv(data_path)
naruto_transcript_df.head()

,name,line
0,Naruto,(Laughing) Give it up. (Shows the stone faces...
1,Hiruzen,(Turns away from his writing) I hope you’re n...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [3]:
# Remove actions from transcript
def remove_paranthesis(text):
  result = re.sub(r'\(.*?\)', '', text)
  return result
 
naruto_transcript_df['line'] = naruto_transcript_df['line'].apply(remove_paranthesis)
naruto_transcript_df.head()   

,name,line
0,Naruto,"Give it up. You’re just bent, because you d..."
1,Hiruzen,I hope you’re not bothering me with some tri...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [4]:
naruto_transcript_df['number_of_words'] = naruto_transcript_df['line'].str.strip().str.split(" ").apply(len)
naruto_transcript_df.head() 

,name,line,number_of_words
0,Naruto,"Give it up. You’re just bent, because you d...",26
1,Hiruzen,I hope you’re not bothering me with some tri...,16
2,Ninja,Naseer Sabah,2
3,Ninja,is the best person on earth,6
4,Naruto,muah,1


In [5]:
naruto_transcript_df['naruto_response_flag'] = 0
naruto_transcript_df.loc[(naruto_transcript_df['name'] == "Naruto") & (naruto_transcript_df['number_of_words'] > 5), 'naruto_response_flag'] = 1

In [8]:
indexes_to_take = list(naruto_transcript_df[(naruto_transcript_df['naruto_response_flag'] == 1) & (naruto_transcript_df.index>0)].index)

In [10]:
indexes_to_take[:4]

[6, 28, 30, 32]

In [12]:
system_prompt = """" You are Naruto from the anime "Naruto". Your response should reflect his personality and speech patterns \n""" 

prompts = []

for ind in indexes_to_take:
  prompt = system_prompt
  
  prompt += naruto_transcript_df.iloc[ind - 1]['line'] 
  prompt += "\n"
  prompt += naruto_transcript_df.iloc[ind]['line']
  prompts.append(prompt)  

In [16]:
print(prompts[7])

" You are Naruto from the anime "Naruto". Your response should reflect his personality and speech patterns 
 We will now start the final exam. When your name is called, proceed to the testing room. The final test will be on... The Clone Jutsu.
  Ugh! That’s my worst technique! Great. I’m never gonna pass!


In [21]:
df = pd.DataFrame({'prompt': prompts})
df.head()

,prompt
0,""" You are Naruto from the anime ""Naruto"". Your..."
1,""" You are Naruto from the anime ""Naruto"". Your..."
2,""" You are Naruto from the anime ""Naruto"". Your..."
3,""" You are Naruto from the anime ""Naruto"". Your..."
4,""" You are Naruto from the anime ""Naruto"". Your..."


In [25]:
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['prompt'],
    num_rows: 21
})